# Import libraries/data

In [ ]:
#Import libraries
import pandas as pd
import thingspeak
import json
import numpy
from keras.models import Sequential
import keras
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [3]:
#Import and clean control data
lenoptions = dict()
lenoptions["results"] = 400

channel = thingspeak.Channel(1663562)
secondsdata = channel.get_field(field=1, options=lenoptions)
cdata = channel.get_field(field=2, options=lenoptions)
cmdata = channel.get_field(field=3, options=lenoptions)
tempdata = channel.get_field(field=4, options=lenoptions)
humiditydata = channel.get_field(field=5, options=lenoptions)

seconds = []
carbond = []
carbonm = []
temp = []
humidity = []
fire = []

def cleandata(data, field, datatype):
    data = json.loads(data)
    for i in data['feeds']:
        datatype.append(i['field' + str(field)])
    
cleandata(secondsdata, 1, seconds)
cleandata(cdata, 2, carbond)
cleandata(cmdata, 3, carbonm)
cleandata(tempdata, 4, temp)
cleandata(humiditydata, 5, humidity)
i = 0
while i < len(secondsdata):
    i += 1
    fire.append(0)
controldf = pd.DataFrame(list(zip(seconds, carbond, carbonm, temp, humidity, fire)), columns = ['Seconds', 'C02 (PPM)', 'CO (PPM)', 'Temperature', 'Humidity', 'Fire'])

#Import and clean fire data
channel = thingspeak.Channel(1665581)
secondsdataf = channel.get_field(field=1)
cdataf = channel.get_field(field=2)
cmdataf = channel.get_field(field=3)
tempdataf = channel.get_field(field=4)
humiditydataf = channel.get_field(field=5)

secondsf = []
carbondf = []
carbonmf = []
tempf = []
humidityf = []
firef = []
cleandata(secondsdataf, 1, secondsf)
cleandata(cdataf, 2, carbondf)
cleandata(cmdataf, 3, carbonmf)
cleandata(tempdataf, 4, tempf)
cleandata(humiditydataf, 5, humidityf)

#Merge control and fire data
i = 0
while i < len(secondsdataf):
    i += 1
    firef.append(1)
firedf = pd.DataFrame(list(zip(secondsf, carbondf, carbonmf, tempf, humidityf, firef)), columns = ['Seconds', 'C02 (PPM)', 'CO (PPM)', 'Temperature', 'Humidity', 'Fire'])

#Print first 5 rows of dataset
df = pd.concat([controldf, firedf])
df.head()

,Seconds,C02 (PPM),CO (PPM),Temperature,Humidity,Fire
0,0,492.0384216308594,0.5013868662507481,18.852901458740234,53.9642333984375,0
1,12,485.2566833496094,0.5013868662507481,18.711376190185547,54.5257568359375,0
2,24,492.6656494140625,0.5013868662507481,18.20401382446289,56.04095458984375,0
3,37,437.8020935058594,0.5013868662507481,17.782100677490234,57.9833984375,0
4,49,478.41339111328125,0.5013868662507481,17.62722396850586,59.20867919921875,0


In [4]:
#Create input and output
x = df.drop(['Fire'], axis = 1)
y = df['Fire']
print(x)
print(y.shape)

   Seconds           C02 (PPM)            CO (PPM)         Temperature  \
0        0   492.0384216308594  0.5013868662507481  18.852901458740234   
1       12   485.2566833496094  0.5013868662507481  18.711376190185547   
2       24   492.6656494140625  0.5013868662507481   18.20401382446289   
3       37   437.8020935058594  0.5013868662507481  17.782100677490234   
4       49  478.41339111328125  0.5013868662507481   17.62722396850586   
..     ...                 ...                 ...                 ...   
43     216   624.5687866210938   0.528156973824964  17.103836059570312   
44     228    624.929443359375   0.528156973824964  17.173267364501953   
45     240   623.9913330078125   0.528156973824964  17.173267364501953   
46     253   630.9193115234375   0.528156973824964  17.189289093017578   
47     265    636.283935546875   0.528156973824964  17.215991973876953   

             Humidity  
0    53.9642333984375  
1    54.5257568359375  
2   56.04095458984375  
3       57.9833

# Data Pre-Processing

In [5]:
#Scale data to make units comparable
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

#Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, random_state=0)

# K-Nearest Neighbors

In [7]:
#Import K-Nearest Neighbors
knn = KNeighborsClassifier()

#Define range for finding optimal neighbors value
k_range = list(range(5, 20))

params = dict(n_neighbors=k_range)
  
#Define parameters for grid search
grid = GridSearchCV(knn, params, cv=10, scoring='accuracy', return_train_score=False, verbose=1)
  
#Perform grid search
grid_search = grid.fit(x_train, y_train)

print((grid_search.best_params_))

#Define parameters for KNN
knn = KNeighborsClassifier(n_neighbors=int(5))

#Fit KNN to training data
knn.fit(x_train, y_train)

#Test on testing set and print accuracy
y_pred = knn.predict(x_test)
print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))

Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'n_neighbors': 5}
Accuracy:  1.0


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.6s finished


# Sequential Neural Network

In [8]:
callback = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=3)
#Define sequential neural network
model = Sequential()

#Add hidden layer
model.add(Dense(4, activation='relu', input_dim=5))

#Add dropout layer
model.add(Dropout(0.1))

#Add layer to classify
model.add(Dense(1, activation='sigmoid'))

#Compile model parameters
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Fit model to training data, tune epoch/batch parameters
history = model.fit(x_train, y_train, epochs=30, batch_size=15, callbacks=[callback], verbose=0)

#Print loss 
print(len(history.history['loss']))

#Test on training data and print loss and accuracy metrics
loss, accuracy = model.evaluate(x_test, y_test)
print("Loss: " + str(loss))
print("Accuracy: " + str(accuracy))

29
3/3 [==============================] - 0s 2ms/step - loss: 0.1721 - accuracy: 0.9726
Loss: 0.17207829654216766
Accuracy: 0.9726027250289917
